<a href="https://colab.research.google.com/github/badonyi/adventofcode2024/blob/main/aov2024_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Run this chunk before any of the daily challenges

In [ ]:
options(scipen = 999)
get_data <- function(x) sprintf('https://raw.githubusercontent.com/badonyi/adventofcode2024/refs/heads/main/data/%s.txt', x)

## Day 1

In [ ]:
input <- read.table(get_data('day_1'))

# part 1
sum(abs(sort(input$V1) - sort(input$V2)))

# part 2
sum(outer(input$V1, input$V2, `==`) * input$V1)

## Day 2

In [ ]:
input <- sapply(strsplit(readLines(get_data('day_2')), ' '), as.integer)

# part 1
sum(sapply(input, function(x) {
  d <- diff(x); all(abs(d) < 4) && (all(d < 0) || all(d > 0))
}))

# part 2
sum(sapply(input, function(x) {
  any(sapply(seq_along(x), function(i) {
    d <- diff(x[-i]); all(abs(d) < 4) && (all(d < 0) || all(d > 0))
  }))
}))


## Day 3

In [ ]:
input <- paste(readLines(get_data('day_3')), collapse = '')
pat <- 'mul\\((\\d+),(\\d+)\\)'
mul_compute <- function(s) {
  eval(parse(text = paste(
    gsub(pat, '\\1*\\2', regmatches(s, gregexpr(pat, s))[[1]]), collapse = '+'
  )))
}

# part 1
mul_compute(input)

# part 2
donts <- strsplit(input, "don't\\(\\)")[[1]]
dos <- unlist(lapply(donts[-1], function(x) strsplit(x, 'do\\(\\)')[[1]][-1]))
mul_compute(paste(c(donts[1], dos), collapse = ''))

## Day 4

In [ ]:
mat <- do.call(rbind, strsplit(readLines(get_data('day_4')), NULL))
has_xmas <- function(x) paste(x, collapse = '') %in% c('XMAS', 'SAMX')

# part 1
n <- nrow(mat)
count <- 0
for (j in seq(n)) {
  for (i in seq(n)) {
    if (i <= n - 3) {
      count <- count + has_xmas(mat[j, i:(i + 3)])
      if (j < n - 2) count <- count + has_xmas(diag(mat[j:(j + 3), i:(i + 3)]))
      if (j > 3) count <- count + has_xmas(diag(mat[j:(j - 3), i:(i + 3)]))
    }
    if (j < n - 2) count <- count + has_xmas(mat[j:(j + 3), i])
  }
}
cat(count)

# part 2
sum(apply(
  X = expand.grid(seq(n), seq(n)),
  MARGIN = 1,
  FUN = function(g) {
    a <- g[1]
    b <- g[2]
    if (a > 1 && a < n && b > 1 && b <  n) {
      return(as.numeric(
        paste0(mat[a - 1, b - 1], mat[a - 1, b + 1],
               mat[a, b], mat[a + 1, b - 1],
               mat[a + 1, b + 1]) %in% c('SMASM', 'MSAMS', 'MMASS', 'SSAMM')))
    }
    return(0)
  }
))

## Day 5

In [ ]:
input <- readLines(get_data('day_5'))
sep <- which(input == '')
updates <- strsplit(input[(sep + 1):length(input)], ',')
pairs <- read.table(text = input[1:(sep - 1)], sep = '|')

is_correct <- function(update, pairs) {
  for (i in 1:nrow(pairs)) {
    if (pairs$V1[i] %in% update && pairs$V2[i] %in% update) {
      if (which(update == pairs$V1[i]) > which(update == pairs$V2[i])) {
        return(FALSE)
      }
    }
  }
  return(TRUE)
}

# part 1
sum(sapply(updates[sapply(updates, is_correct, pairs)],
           function(x) as.numeric(x[ceiling(length(x) / 2)])))

# part 2
correct_order <- function(update, pairs) {
  swap <- TRUE
  while (swap) {
    swap <- FALSE
    for (i in 1:nrow(pairs)) {
      if (pairs$V1[i] %in% update && pairs$V2[i] %in% update) {
        b_index <- which(update == pairs$V1[i])
        a_index <- which(update == pairs$V2[i])
        if (b_index > a_index) {
          update[c(a_index, b_index)] <- update[c(b_index, a_index)]
          swap <- TRUE
        }
      }
    }
  }
  return(update)
}

updates_wrong <- updates[!sapply(updates, is_correct, pairs)]
updates_right <- lapply(updates_wrong, correct_order, pairs)
sum(sapply(updates_right, function(x) as.numeric(x[ceiling(length(x) / 2)])))

## Day 6

In [ ]:
input <- do.call(rbind, strsplit(readLines(get_data('day_6')), NULL))
start <- which(input == '^', arr.ind = TRUE)
dirs <- list(c(-1, 0), c(0, 1), c(1, 0), c(0, -1))
cache <- new.env(hash = TRUE, parent = emptyenv())

explore_grid <- function(grid, cur_pos, obstacle = NULL) {
  state <- if (!is.null(obstacle)) {
     paste(obstacle[1], obstacle[2], sep = ',')
  } else {
    'init'
  }

  if (exists(state, envir = cache)) return(get(state, envir = cache))
  if (!is.null(obstacle)) grid[obstacle[1], obstacle[2]] <- '#'

  visited <- matrix(0, nrow = nrow(grid), ncol = ncol(grid))
  cur_dir <- 1
  visited[cur_pos[1], cur_pos[2]] <- cur_dir

  while (TRUE) {
    new_pos <- cur_pos + dirs[[cur_dir]]
    if (new_pos[1] < 1 || new_pos[1] > nrow(grid) ||
        new_pos[2] < 1 || new_pos[2] > ncol(grid)) {
      result <- if (is.null(obstacle)) visited else FALSE
      assign(state, result, envir = cache)
      return(result)
    }

    if (grid[new_pos[1], new_pos[2]] == '#') {
      cur_dir <- (cur_dir %% 4) + 1
    } else {
      if (visited[new_pos[1], new_pos[2]] == cur_dir) {
        result <- if (is.null(obstacle)) visited else TRUE
        assign(state, result, envir = cache)
        return(result)
      }
      visited[new_pos[1], new_pos[2]] <- cur_dir
      cur_pos <- new_pos
    }
  }
}

# part 1
sum(explore_grid(input, start) != 0)

# part 2
all_loc <- which(input == '.', arr.ind = TRUE)
loop_count <- 0
pb <- txtProgressBar(min = 0, max = nrow(all_loc), style = 3)
for (i in seq_len(nrow(all_loc))) {
  loop_count <- loop_count + explore_grid(input, start, all_loc[i, ])
  setTxtProgressBar(pb, i)
}
close(pb)
loop_count

## Day 7

In [ ]:
permutations <- function(n, r, v, repeats.allowed = FALSE) {
  if (r == 0) return(matrix(nrow = 1, ncol = 0))
  if (r == 1) return(matrix(v, ncol = 1))

  result <- NULL
  for (i in seq_along(v)) {
    if (repeats.allowed || !(v[i] %in% v[1:(i - 1)])) {
      sub_perms <- permutations(n, r - 1, v, repeats.allowed)
      perms <- cbind(matrix(v[i], nrow = nrow(sub_perms), ncol = 1), sub_perms)
      result <- rbind(result, perms)
    }
  }
  return(result)
}

run_ops <- function(test_val, num_val, ops) {
  eval_ops <- function(current, remaining, test_val) {
    if (length(remaining) == 0) return(current == test_val)

    for (op in ops) {
      next_num <- remaining[1]
      new_current <- switch(op,
                            '+' = current + next_num,
                            '*' = current * next_num,
                            '||' = as.numeric(paste0(current, next_num)))
      if (eval_ops(new_current, remaining[-1], test_val)) return(TRUE)
    }
    return(FALSE)
  }

  if (length(num_val) == 1) return(ifelse(test_val == num_val[1], test_val, 0))
  if (eval_ops(num_val[1], num_val[-1], test_val)) return(test_val)
  return(0)
}

input <- strsplit(readLines(get_data('day_7')), ': ')
test_val <- as.numeric(sapply(input, `[`, 1))
num_val <- lapply(sapply(input, `[`, 2), function(x) {
  as.numeric(strsplit(x, ' ')[[1]])
})

# part 1
sum(sapply(seq_along(test_val), function(i) {
  run_ops(test_val[i], num_val[[i]], c('+', '*'))
}))

# part 2
sum(sapply(seq_along(test_val), function(i) {
  run_ops(test_val[i], num_val[[i]], c('+', '*', '||'))
}))

## Day 8

In [ ]:
input <- readLines(get_data('day_8'))
mat <- do.call(rbind, strsplit(input, ''))
antennae <- unique(mat[mat != '.'])

get_antinode <- function(mat, freq, extend = FALSE) {
  pos <- which(mat == freq, arr.ind = TRUE)
  if (nrow(pos) < 2) return(matrix(NA, nrow = 0, ncol = 2))
  combs <- combn(seq_len(nrow(pos)), 2)

  results <- NULL
  for (i in seq_len(ncol(combs))) {
    pos1 <- pos[combs[1, i], ]
    pos2 <- pos[combs[2, i], ]
    dist <- pos1 - pos2

    if (extend) {
      for (k in seq(0, ncol(mat))) {
        results <- rbind(results, pos1 + k * dist, pos2 - k * dist)
      }
    } else {
      results <- rbind(results, pos1 + dist, pos2 - dist)
    }
  }

  unique(results[results[, 1] > 0 & results[, 1] <= nrow(mat) &
                 results[, 2] > 0 & results[, 2] <= ncol(mat), ])
}

# part 1
nrow(unique(do.call(
  rbind, lapply(antennae, function(freq)
    get_antinode(mat, freq, extend = FALSE))
)))

# part 2
nrow(unique(do.call(
  rbind, lapply(antennae, function(freq)
    get_antinode(mat, freq, extend = TRUE))
)))

## Day 9

In [ ]:
input <- readLines(get_data('day_9'))

# part 1
digits <- as.numeric(strsplit(input, '')[[1]])
ids <- (seq(digits) - 1) / 2
disk_long <- ids[rep(seq(digits), digits)]
files_to_move <- rev(disk_long[disk_long == floor(disk_long)])
empty_idx <- which(disk_long != floor(disk_long))
disk_long[empty_idx] <- files_to_move[seq(empty_idx)]
sum(disk_long[seq(files_to_move)] * (seq(files_to_move) - 1))

# part 2
blocks <- mapply(rep, times = digits, x = ifelse((!seq(digits) %% 2), -1, ids))
free_space <- digits * (!seq(digits) %% 2)

for (file_id in seq(length(blocks), 2, by = -1)) {
  file <- blocks[[file_id]][blocks[[file_id]] != -1]

  if (length(file) > 0) {
    blocks[[file_id]][] <- -1
    free_space[file_id] <- free_space[file_id] + length(file)
    block <- which(free_space >= length(file))[1]

    if (!is.na(block)) {
      blocks[[block]][blocks[[block]] == -1][seq_along(file)] <- file
      free_space[block] <- free_space[block] - length(file)
    }
  }
}

compact_blocks <- unlist(blocks[digits != 0])
sum((compact_blocks * (compact_blocks > 0)) * (seq(compact_blocks) - 1))

## Day 10

In [ ]:
input <- readLines(get_data('day_10'))
topo_map <- do.call(rbind, lapply(strsplit(input, ''), as.integer))
height <- nrow(topo_map)
width <- ncol(topo_map)
trailheads <- which(topo_map == 0, arr.ind = TRUE)
moves <- list(up = c(-1, 0), down = c(1, 0), left = c(0, -1), right = c(0, 1))

explore_paths <- function(row, col, elevation, visited, end, path, paths) {
  if (row <= 0 || row > height || col <= 0 || col > width ||
      topo_map[row, col] != elevation || visited[row, col]) {
    return(list(end = end, paths = paths))
  }

  path <- rbind(path, c(row, col))

  if (elevation == 9) {
    end <- unique(rbind(end, c(row, col)))
    paths <- c(paths, list(path))
    return(list(end = end, paths = paths))
  }

  visited[row, col] <- TRUE

  result <- lapply(moves, function(move) {
    explore_paths(row + move[1], col + move[2],
                  elevation + 1, visited, end, path, paths)
  })

  end <- do.call(rbind, lapply(result, `[[`, 'end'))
  paths <- do.call(c, lapply(result, `[[`, 'paths'))

  visited[row, col] <- FALSE

  return(list(end = unique(end), paths = paths))
}

visited <- matrix(FALSE, nrow = height, ncol = width)
results <- apply(trailheads, 1, function(start) {
  explore_paths(start[1], start[2], 0, visited,
                end = matrix(nrow = 0, ncol = 2),
                path = matrix(nrow = 0, ncol = 2),
                paths = vector(mode = 'list'))
})

# part 1
sum(sapply(results, function(res) nrow(res$end)))

# part 2
all_paths <- do.call(c, lapply(results, `[[`, 'paths'))
length(unique(sapply(all_paths, function(path) {
  paste(apply(path, 1, paste, collapse = ','), collapse = ';')
})))

## Day 11

In [ ]:
transform_stone <- function(stone) {
  chr <- as.character(stone)
  if (stone == 0) {
    return(1)
  } else if (nchar(chr) %% 2 == 0) {
    return(
      c(
        as.numeric(substr(chr, 1, nchar(chr) / 2)),
        as.numeric(substr(chr, nchar(chr) / 2 + 1, nchar(chr)))
      )
    )
  } else {
    return(stone * 2024)
  }
}

blink <- function(stones, times) {
  tbl <- table(stones)

  for (i in seq(times)) {
    tmp <- unlist(lapply(names(tbl), function(idx) {
      count <- tbl[idx]
      new_stones <- transform_stone(as.numeric(idx))
      rep(count, length(new_stones))
    }), use.names = FALSE)

    names(tmp) <- unlist(lapply(names(tbl), function(idx) {
      new_stones <- transform_stone(as.numeric(idx))
      as.character(new_stones)
    }))

    tbl <- tapply(tmp, names(tmp), sum)
  }
  tbl
}

# part 1
stones <- scan(get_data('day_11'), numeric())
sum(blink(stones, 25))

# part 2
sum(blink(stones, 75))

## Day 12

In [ ]:
input <- strsplit(readLines(get_data('day_12')), '')
grid <- do.call(rbind, input)

count_sides <- function(seen) {
  pad <- rbind(FALSE, cbind(FALSE, seen, FALSE), FALSE)
  count_sides <- function(seen) {
    sides <- 0
    for (i in seq(nrow(seen) - 1)) {
      row1 <- seen[i, ]
      row2 <- seen[i + 1, ]
      sides <- sides + sum(diff(c(FALSE, row1 & !row2)) == 1) +
        sum(diff(c(FALSE, row2 & !row1)) == 1)
    }
    sides
  }
  return(count_sides(pad) + count_sides(t(pad)))
}

block_price <- function(grid, coord, sides = FALSE) {
  dir <- list(c(-1, 0), c(1, 0), c(0, -1), c(0, 1))
  seen <- matrix(FALSE, nrow = nrow(grid), ncol = ncol(grid))
  block <- grid[coord[1], coord[2]]
  coords <- list(coord)
  area <- perim <- 0

  while (length(coords) > 0) {
    idx <- matrix(coords[[1]], nrow = 1)
    coords <- coords[-1]

    if (any(idx <= 0) | any(idx > dim(grid))) {
      perim <- perim + 1
      next
    }

    if (grid[idx] != block) {
      perim <- perim + 1
      next
    }

    if (seen[idx]) {
      next
    } else {
      seen[idx] <- TRUE
    }

    area <- area + 1
    coords <- c(coords, lapply(dir, function(x) idx + x))
  }

  if (sides) {
    perim <- count_sides(seen)
  }

  grid[seen] <- 'seen'
  return(list(grid = grid, price = perim * area))
}

calculate_price <- function(grid, sides = FALSE) {
  price <- 0
  while (length(block_coord <- which(grid != 'seen', arr.ind = TRUE)) > 0) {
    coord <- block_coord[1, , drop = FALSE]
    block <- block_price(grid, coord, sides)
    grid <- block$grid
    price <- price + block$price
  }
  return(price)
}

# part 1
calculate_price(grid, sides = FALSE)

# part 2
calculate_price(grid, sides = TRUE)

## Day 13

In [ ]:
input <- lapply(
  strsplit(paste0(readLines(get_data('day_13')), collapse = '\n'), '\n\n')[[1]],
  function(x) as.numeric(unlist(regmatches(x, gregexpr('[+\\-]?\\d+', x))))
)

start_claw <- function(part2 = FALSE) {
  sum(sapply(input, function(i) {
    x <- matrix(i[1:4], 2)
    y <- matrix(i[5:6], 2) + if (part2) 1e13 else 0
    z <- solve(x, y)
    if (all(x %*% round(z) == y) && all(z > 0) && (all(z <= 1e2) || part2))
      sum(z * c(3, 1)) else 0
  }))
}

# part 1
start_claw()

# part 2
start_claw(part2 = TRUE)

## Day 14

In [ ]:
input <- readLines(get_data('day_14'))
input <- lapply(regmatches(input, gregexpr('-?\\d+', input)), as.numeric)

safety_val <- function(t) {
  final_positions <- function(t) {
    do.call(rbind, lapply(input, function(r) {
      px <- r[1]; py <- r[2]; vx <- r[3]; vy <- r[4]
      c((px + vx * t) %% 101, (py + vy * t) %% 103)
    }))
  }

  prod(table(apply(final_positions(t), 1, function(xy) {
    x <- xy[1]; y <- xy[2]; mx <- 101 %/% 2; my <- 103 %/% 2
    if (x < mx && y < my) return(1)
    if (x > mx && y < my) return(2)
    if (x < mx && y > my) return(3)
    if (x > mx && y > my) return(4)
    return(NA)
  })))
}

# part 1
safety_val(100)

# part 2
which.min(sapply(seq.int(1, 1e4), function(t) safety_val(t)))

## Day 15

In [ ]:
input <- readLines(get_data('day_15'))
sep <- which(input == '')
grid <- do.call(rbind, strsplit(input[1:(sep - 1)], ''))
moves <- unlist(strsplit(input[(sep + 1):length(input)], ''))

try_move1 <- function(p, d) {
  if (d == -1) return(rev(try_move1(rev(p), 1)))

  pos <- which(p == '@')
  if (p[pos + 1] == '#') return(p)
  if (p[pos + 1] == '.') { p[pos] <- '.'; p[pos + 1] <- '@'; return(p) }

  path_ahead <- p[seq((pos + 1), length(p))]
  path_ahead <- path_ahead[seq_len(min(which(path_ahead == '#')))]
  if (any(path_ahead == '.')) {
    p[pos] <- '.'
    pos <- pos + 1
    p[pos] <- '@'
    next_dot_pos <- min(which(p[seq((pos + 1), length(p))] == '.')) + pos
    if (length(next_dot_pos) > 0) p[next_dot_pos] <- 'O'
  }

  return(p)
}

try_move2 <- function(grid, move) {
  robot <- which(grid == '@', arr.ind = TRUE)
  visited <- matrix(FALSE, nrow = nrow(grid), ncol = ncol(grid))
  moves_2 <- list('>' = c(0, 1), '<' = c(0, -1), '^' = c(-1, 0), 'v' = c(1, 0))
  m <- moves_2[[move]]

  if (grid[robot + m] == '.') {
    grid[robot] <- '.'
    grid[robot + m] <- '@'
    return(grid)
  }

  if (grid[robot + m] == '#') return(grid)

  queue <- vector(mode = 'list')
  queue <- append(queue, list(robot))
  while (length(queue) > 0) {

    pos <- queue[[1]]
    queue <- queue[-1]
    if (visited[pos[1], pos[2]]) next

    visited[pos[1], pos[2]] <- TRUE

    new_pos <- pos + m
    if (grid[new_pos] == '#') return(grid)

    if (!is.na(grid[new_pos[1], new_pos[2]]) &&
        grid[new_pos[1], new_pos[2]] == '[') {
      queue <- append(queue, list(new_pos))
      queue <- append(queue, list(new_pos + c(0, 1)))
    }

    if (!is.na(grid[new_pos[1], new_pos[2]]) &&
        grid[new_pos[1], new_pos[2]] == ']') {
      queue <- append(queue, list(new_pos))
      queue <- append(queue, list(new_pos + c(0, -1)))
    }

  }

  moved_boxes <- which(visited & grid %in% c('[',']','@'), arr.ind = TRUE)
  boxes <- grid[moved_boxes]
  grid[moved_boxes] <- '.'
  moved_boxes <- t(t(moved_boxes) + m)
  grid[moved_boxes] <- boxes

  return(grid)
}

move_robot <- function(grid, moves, part2 = FALSE) {
  for (move in moves) {
    pos <- which(grid == '@', arr.ind = TRUE)

    if (part2) {
      grid <- try_move2(grid, move)
    } else {
      if (move == '>') {
        grid[pos[1], ] <- try_move1(grid[pos[1], ], 1)
      } else if (move == '<') {
        grid[pos[1], ] <- try_move1(grid[pos[1], ], -1)
      } else if (move == '^') {
        grid[, pos[2]] <- try_move1(grid[, pos[2]], -1)
      } else if (move == 'v') {
        grid[, pos[2]] <- try_move1(grid[, pos[2]], 1)
      }
    }
  }
  return(grid)
}

gps_score <- function(grid, part2 = FALSE) {
  box <- if (part2) '[' else 'O'
  boxes <- which(grid == box, arr.ind = TRUE) - 1
  if (nrow(boxes) == 0) return(0)
  sum(apply(boxes, 1, function(x) 100 * x[1] + x[2]))
}

# part 1
gps_score(move_robot(grid, moves))

# part 2
grid2 <-  {
  chr <- list(
    '#' = c('#', '#'),
    'O' = c('[', ']'),
    '.' = c('.', '.'),
    '@' = c('@', '.')
  )
  t(apply(grid, 1, function(r) unlist(lapply(r, function(i) chr[[i]]))))
}

gps_score(move_robot(grid2, moves, part2 = TRUE), part2 = TRUE)

## Day 16

In [ ]:
input <- readLines(get_data('day_16'))
maze <- do.call(rbind, sapply(input, strsplit, NULL, USE.NAMES = FALSE))
start <- which(maze == 'S', arr.ind = TRUE)
end <- which(maze == 'E', arr.ind = TRUE)
directions <- list(U = c(-1, 0), R = c(0, 1), D = c(1, 0), L = c(0, -1))

rotate <- function(current_dir, turn_side) {
  dirs <- c('U', 'R', 'D', 'L')
  idx <- match(current_dir, dirs)
  idx <- if (turn_side == 'L') idx - 1 else idx + 1
  idx <- ifelse(idx < 1, 4, ifelse(idx > 4, 1, idx))
  dirs[idx]
}

get_moves <- function(state) {
  moves <- list()
  next_pos <- state$pos + directions[[state$dir]]
  moves$move <- list(pos = next_pos, dir = state$dir, score = state$score + 1)
  left_dir <- rotate(state$dir, 'L')
  moves$turnL <- list(pos = state$pos + directions[[left_dir]],
                      dir = left_dir, score = state$score + 1001)
  right_dir <- rotate(state$dir, 'R')
  moves$turnR <- list(pos = state$pos + directions[[right_dir]],
                      dir = right_dir, score = state$score + 1001)
  moves
}

min_score <- 0
visited <- array(FALSE, dim = c(nrow(maze), ncol(maze), 4))
queue <- list(list(pos = start, dir = 'R', score = 0))
while (length(queue) > 0) {
  scores <- sapply(queue, `[[`, 'score')
  current <- queue[[which.min(scores)]]
  queue <- queue[-which.min(scores)]

  if (maze[current$pos[1], current$pos[2]] == '#' ||
      visited[current$pos[1], current$pos[2],
              match(current$dir, c('U', 'R', 'D', 'L'))]) next

  visited[current$pos[1], current$pos[2],
          match(current$dir, c('U', 'R', 'D', 'L'))] <- TRUE

  if (all(current$pos == end)) {
    min_score <- current$score
    break
  }

  for (move in get_moves(current)) {
    if (move$pos[1] > 0 && move$pos[1] <= nrow(maze) &&
        move$pos[2] > 0 && move$pos[2] <= ncol(maze) &&
        maze[move$pos[1], move$pos[2]] != '#') {
      queue <- append(queue, list(move))
    }
  }
}

# part 1
cat(min_score)

# part 2
merged_path <- matrix(FALSE, nrow = nrow(maze), ncol = ncol(maze))
min_scores <- array(Inf, dim = c(nrow(maze), ncol(maze), 4))
paths <- list()
queue <- list(list(
  pos = start,
  dir = 'R',
  score = 0,
  path = matrix(FALSE, nrow = nrow(maze), ncol = ncol(maze))
))

while (length(queue) > 0) {
  scores <- sapply(queue, `[[`, 'score')
  current <- queue[[which.min(scores)]]
  queue <- queue[-which.min(scores)]

  if (maze[current$pos[1], current$pos[2]] == '#' ||
      current$score > min_scores[current$pos[1], current$pos[2],
                                 match(current$dir, c('U', 'R', 'D', 'L'))])
    next

  min_scores[current$pos[1], current$pos[2],
             match(current$dir, c('U', 'R', 'D', 'L'))] <- current$score

  current$path[current$pos[1], current$pos[2]] <- TRUE
  if (all(current$pos == end) && current$score == min_score) {
    paths <- append(paths, list(current$path))
    next
  }

  for (move in get_moves(current)) {
    if (move$pos[1] > 0 && move$pos[1] <= nrow(maze) &&
        move$pos[2] > 0 && move$pos[2] <= ncol(maze) &&
        maze[move$pos[1], move$pos[2]] != '#') {
      move$path <- current$path
      queue <- append(queue, list(move))
    }
  }
}

for (path in paths) { merged_path <- merged_path | path }
sum(merged_path)

## Day 17

In [ ]:
input <- readLines(get_data('day_17'))
d <- as.numeric(unlist(regmatches(input, gregexpr('\\d+', input))))
p <- d[-(1:3)]
r_init <- d[1:3]

run_program <- function(program, r_init) {
  r <- r_init
  o <- integer()
  i <- 1
  n <- length(program)
  while (i <= n) {
    opcode <- program[i]
    if (i + 1 > n) break
    operand <- program[i + 1]
    v <- if (operand <= 3) operand else r[operand - 3]
    switch(opcode + 1,
           r[1] <- r[1] %/% 2^v,
           r[2] <- bitwXor(r[2], operand),
           r[2] <- v %% 8,
           if (r[1] != 0) i <- operand + 1 else i <- i + 2,
           r[2] <- bitwXor(r[2], r[3]),
           o <- c(o, v %% 8),
           r[2] <- r[1] %/% 2^v,
           r[3] <- r[1] %/% 2^v)
    if (opcode != 3) i <- i + 2
  }
  o
}

# part 1
paste(run_program(p, r_init), collapse = ',')

# part 2 - could only solve with a long exhaustive search;
# this is an analytical solution by github user bjebert
tr <- function(n, x) {
  if (x < 0 || x > 7) return(NA)
  if (x == 0) return(n)
  trans <- list(
    NULL,
    c(1, -1),
    c(-2, 2, 2, -2),
    c(-1, 1, 3, -3),
    c(-4, -4, -4, 4, 4, 4, 4, -4),
    c(-3, -5, -3, 3, 5, 3, 5, -5),
    c(-6, -2, -2, 2, 2, 6, 6, -6),
    c(-5, -3, -1, 1, 3, 5, 7, -7)
  )
  pat <- trans[[x + 1]]
  return(n - pat[(n - 1) %% length(pat) + 1])
}

a <- 0
for (i in seq.int(length(p), 1)) {
  a_rng <- sapply(a, function(x) (x * 8) + seq.int(0, 7))
  a_rng <- a_rng[a_rng != 0]
  a <- a_rng[sapply(a_rng, function(x) tr(floor(x / 2^tr(x %% 8, 3)),
                                          tr(x %% 8, 3))) %% 8 == tr(p[i], 5)]
}
min(a)

## Day 18

In [ ]:
input <- strsplit(readLines(get_data('day_18')), ',')
memspace <- matrix('.', nrow = 71, ncol = 71)
x <- as.numeric(sapply(input, function(x) x[1])) + 1
y <- as.numeric(sapply(input, function(x) x[2])) + 1
for (i in seq.int(1024)) { memspace[y[i], x[i]] <- '#' }

directions <- list(U = c(-1, 0), R = c(0, 1), D = c(1, 0), L = c(0, -1))

rotate <- function(current_dir, turn_side) {
  dirs <- c('U', 'R', 'D', 'L')
  idx <- match(current_dir, dirs)
  idx <- if (turn_side == 'L') idx - 1 else idx + 1
  idx <- ifelse(idx < 1, 4, ifelse(idx > 4, 1, idx))
  dirs[idx]
}

get_moves <- function(state) {
  moves <- list()
  next_pos <- state$pos + directions[[state$dir]]
  moves$move <- list(pos = next_pos, dir = state$dir, score = state$score + 1)
  left_dir <- rotate(state$dir, 'L')
  moves$turnL <- list(pos = state$pos + directions[[left_dir]],
                      dir = left_dir, score = state$score + 1)
  right_dir <- rotate(state$dir, 'R')
  moves$turnR <- list(pos = state$pos + directions[[right_dir]],
                      dir = right_dir, score = state$score + 1)
  moves
}

min_score <- 0
visited <- array(FALSE, dim = c(nrow(memspace), ncol(memspace), 4))
queue <- list(list(pos = c(1, 1), dir = 'R', score = 0))
while (length(queue) > 0) {
  scores <- sapply(queue, `[[`, 'score')
  current <- queue[[which.min(scores)]]
  queue <- queue[-which.min(scores)]

  if (memspace[current$pos[1], current$pos[2]] == '#' ||
      visited[current$pos[1], current$pos[2],
              match(current$dir, c('U', 'R', 'D', 'L'))]) next

  visited[current$pos[1], current$pos[2],
          match(current$dir, c('U', 'R', 'D', 'L'))] <- TRUE

  if (all(current$pos == c(71, 71))) {
    min_score <- current$score
    break
  }

  for (move in get_moves(current)) {
    if (move$pos[1] > 0 && move$pos[1] <= nrow(memspace) &&
        move$pos[2] > 0 && move$pos[2] <= ncol(memspace) &&
        memspace[move$pos[1], move$pos[2]] != '#') {
      queue <- append(queue, list(move))
    }
  }
}

# part 1
cat(min_score)

# part 2
is_path_blocked <- function(memspace) {
  visited <- matrix(FALSE, nrow = nrow(memspace), ncol = ncol(memspace))
  queue <- list(c(1, 1))
  visited[1, 1] <- TRUE

  while (length(queue) > 0) {
    current <- queue[[1]]
    queue <- queue[-1]

    if (all(current == c(71, 71))) return(FALSE)

    for (d in directions) {
      next_pos <- current + d
      if (next_pos[1] > 0 && next_pos[1] <= nrow(memspace) &&
          next_pos[2] > 0 && next_pos[2] <= ncol(memspace) &&
          !visited[next_pos[1], next_pos[2]] &&
          memspace[next_pos[1], next_pos[2]] == '.') {
        queue <- append(queue, list(next_pos))
        visited[next_pos[1], next_pos[2]] <- TRUE
      }
    }
  }

  return(TRUE)
}

for (i in seq.int(1025, length(x))) {
  memspace[y[i], x[i]] <- '#'
  if (is_path_blocked(memspace)) {
    cat(x[i] - 1, y[i] - 1, sep = ',')
    break
  }
}

## Day 19

In [ ]:
input <- readLines(get_data('day_19'))
towels <- unlist(strsplit(input[1], ', '))
patterns <- input[-c(1, 2)]

solve_pattern <- function(pattern, towels) {
  n <- nchar(pattern)
  dp <- integer(n + 1)
  dp[1] <- 1

  for (i in seq_len(n)) {
    if (dp[i] > 0) {
      for (towel in towels) {
        len_towel <- nchar(towel)
        if (i + len_towel <= n + 1) {
          substring_pattern <- substr(pattern, i, i + len_towel - 1)
          if (substring_pattern == towel) {
            dp[i + len_towel] <- dp[i + len_towel] + dp[i]
          }
        }
      }
    }
  }

  return(dp)
}

# part 1
possible_patterns <- lapply(patterns, function(p) solve_pattern(p, towels))
sum(sapply(possible_patterns, function(x) x[length(x)] > 0))

# part 2
sum(sapply(possible_patterns, function(x) x[length(x)]))

## Day 20

In [ ]:
map <- readLines(get_data('day_20'))
track <- do.call(rbind, strsplit(map, ''))
dirs <- list(c(-1, 0), c(0, 1), c(1, 0), c(0, -1))
start <- which(track == 'S', arr.ind = TRUE)[1, ]
end <- which(track == 'E', arr.ind = TRUE)[1, ]

calc_dist <- function(track) {
  dist <- matrix(Inf, nrow(track), ncol(track))
  dist[start[1], start[2]] <- 0
  queue <- list(start)
  q_front <- 1

  while (q_front <= length(queue)) {
    curr <- queue[[q_front]]
    q_front <- q_front + 1

    for (d in dirs) {
      nxt <- curr + d
      if (nxt[1] < 1 || nxt[1] > nrow(track) || nxt[2] < 1 ||
          nxt[2] > ncol(track) || track[nxt[1], nxt[2]] == '#') next

      if (is.infinite(dist[nxt[1], nxt[2]])) {
        dist[nxt[1], nxt[2]] <- dist[curr[1], curr[2]] + 1
        queue[[length(queue) + 1]] <- nxt
      }
    }
  }
  dist
}

dist <- calc_dist(track)
valid <- which(!is.infinite(dist), arr.ind = TRUE)
pos <- data.frame(row = valid[, 1], col = valid[, 2], d = dist[valid])
comb <- expand.grid(x = seq_len(nrow(pos)), y = seq_len(nrow(pos)))
comb <- comb[comb$x != comb$y, ]
a <- abs(pos$row[comb$x] - pos$row[comb$y]) + abs(pos$col[comb$x] - pos$col[comb$y])
b <- pos$d[comb$y] - pos$d[comb$x] - a

# part 1
sum(a == 2 & b >= 100)

# part 2
sum(a <= 20 & b >= 100)

# Day 21

In [ ]:
input <- readLines(get_data('day_21'))
keypad <- matrix(c(7:9, 4:6, 1:3, '', 0, 'A'), nrow = 4, byrow = TRUE)
dirpad <- matrix(c('', '^', 'A', '<', 'v', '>'), nrow = 2, byrow = TRUE)

cache <- list()
is_cached <- function(key) any(sapply(cache, function(x) identical(x$key, key)))

cache_get <- function(key) {
  cache[which(sapply(cache, function(x) identical(x$key, key)))][[1]]$value
}

cache_add <- function(key, value) {
  cache <<- c(cache, list(list(key = key, value = value)))
}

move <- function(start, end, is_keypad = FALSE) {
  pad <- if (is_keypad) keypad else dirpad
  diff <- end - start
  v_move <- if (diff[1] > 0) 'v' else '^'
  v <- rep(v_move, abs(diff[1]))
  h_move <- if (diff[2] > 0) '>' else '<'
  h <- rep(h_move, abs(diff[2]))
  if (diff[2] > 0 && pad[end[1], start[2]] != '') return(c(v, h, 'A'))
  if (pad[start[1], end[2]] != '') return(c(h, v, 'A'))
  if (pad[end[1], start[2]] != '') return(c(v, h, 'A'))
}

get_moves <- function(s, curr_pos, pad, is_keypad = FALSE) {
  s_out <- c()
  for (char in s) {
    pos <- which(pad == char, arr.ind = TRUE)[1, ]
    s_out <- c(s_out, move(curr_pos, pos, is_keypad))
    curr_pos <- pos
  }
  s_out
}

get_dir_moves <- function(s) {
  get_moves(s[-1], which(dirpad == s[1], arr.ind = TRUE), dirpad, FALSE)
}

get_total_moves <- function(s, n) {
  s <- c('A', s)
  sn <- length(s) - 1
  if (is_cached(list(s, n))) return(cache_get(list(s, n)))
  if (n == 1) {
    num <- length(get_dir_moves(s))
  } else {
    num <- 0
    for (i in seq.int(1, sn)) {
      num <- num + get_total_moves(get_dir_moves(s[c(i, i + 1)]), n - 1)
    }
  }
  cache_add(list(s, n), num)
  num
}

a <- b <- 0
for (c in input) {
  num <- as.numeric(gsub('[^[:digit:]]', '', c))
  moves <- get_moves(strsplit(c, '')[[1]], c(4, 3), keypad, TRUE)
  a <- a + num * get_total_moves(moves, 2)
  b <- b + num * get_total_moves(moves, 25)
}
cat('Part 1:', a, '\nPart 2:', b)

## Day 22

In [ ]:
input <- scan(get_data('day_22'), what = integer())

next_secret <- function(secret) {
  prune <- function(x) x %% 16777216
  secret <- bitwXor(secret, prune(secret * 64))
  secret <- bitwXor(secret, prune(secret %/% 32))
  secret <- bitwXor(secret, prune(secret * 2048))
  prune(secret)
}

nth_secret <- function(secret, n) {
  for (i in seq.int(1, n)) { secret <- next_secret(secret) }
  secret
}

# part 1
sum(sapply(input, nth_secret, n = 2000))

# part 2
find_best_sequence <- function(secrets, max_steps = 2000) {
  get_prices <- function(secret, steps) {
    prices <- integer(steps)
    for (i in seq_len(steps)) {
      secret <- next_secret(secret)
      prices[i] <- secret %% 10
    }
    prices
  }

  pattern_map <- new.env(hash = TRUE, parent = emptyenv())
  for (secret in secrets) {
    prices <- get_prices(secret, max_steps)
    changes <- diff(prices)

    seen_patterns <- new.env(hash = TRUE, parent = emptyenv())
    for (i in seq_len(length(changes) - 3)) {
      pattern <- paste(changes[i:(i + 3)], collapse = ',')
      value <- prices[i + 4]

      if (is.null(seen_patterns[[pattern]])) {
        seen_patterns[[pattern]] <- TRUE

        if (!is.null(pattern_map[[pattern]])) {
          pattern_map[[pattern]] <- pattern_map[[pattern]] + value
        } else {
          pattern_map[[pattern]] <- value
        }
      }
    }
  }

  pattern_keys <- ls(pattern_map)
  pattern_values <- sapply(pattern_keys, function(key) pattern_map[[key]])
  unname(pattern_values[which.max(pattern_values)])
}

find_best_sequence(input)

## Day 23

In [ ]:
input <- strsplit(readLines(get_data('day_23')), '-')

# part 1
connections <- list()
for (c in input) {
  connections[[c[1]]] <- c(connections[[c[1]]], c[2])
  connections[[c[2]]] <- c(connections[[c[2]]], c[1])
}

t_conn <- connections[startsWith(names(connections), 't')]

part_1 <- list()
for (i in seq_along(t_conn)) {
  tmp <- list()
  for (j in t_conn[[i]]) {
    for (k in intersect(connections[[j]], t_conn[[i]])) {
      tmp <- c(tmp, list(c(names(t_conn)[i], j, k)))
    }
  }
  part_1[[i]] <- lapply(tmp, sort)
}

length(unique(lapply(unlist(part_1, recursive = FALSE), sort)))

# part 2
computers <- names(connections)
longest <- c()
for (comp in computers) {
  queue <- seen <- list()
  for (adj in connections[[comp]]) queue <- append(queue, adj)

  cur_cliq <- comp
  while (length(queue) > 0) {
    c2 <- queue[[1]]
    queue <- queue[-1]

    if (!is.null(seen[[c2]])) next
    seen[[c2]] <- TRUE

    new_pos <- connections[[c2]]
    if (all(cur_cliq %in% new_pos)) {
      cur_cliq <- c(cur_cliq, c2)
      for (item in new_pos) queue <- append(queue, item)
    }
  }

  cur_cliq <- sort(cur_cliq)
  if (length(cur_cliq) > length(longest)) longest <- cur_cliq
}

paste0(longest, collapse = ',')

## Day 24

In [ ]:
lines <- strsplit(paste(readLines(get_data('day_24')), collapse = '\n'), '\n')[[1]]
init <- lines[grepl(':', lines)]
wires <- setNames(as.numeric(sub('.*: ', '', init)), sub(':.*', '', init))
gate_lines <- lines[!grepl(':', lines) & lines != '']

gates <- lapply(gate_lines, function(line) {
  parts <- strsplit(line, ' ')[[1]]
  list(inp1 = parts[1], op = parts[2], inp2 = parts[3], output = parts[5])
})

apply_gate_operation <- function(op, x, y) {
  switch(
    op,
    AND = as.numeric(x & y),
    OR = as.numeric(x | y),
    XOR = as.numeric(x != y)
  )
}

process_gates <- function(wires, gates) {
  while (length(gates) > 0) {
    for (i in seq_along(gates)) {
      gate <- gates[[i]]
      inputs <- c(gate$inp1, gate$inp2)

      if (all(inputs %in% names(wires))) {
        wires[gate$output] <-
          apply_gate_operation(gate$op, wires[gate$inp1], wires[gate$inp2])
        gates[[i]] <- NULL
        break
      }
    }
    gates <- gates[!sapply(gates, is.null)]
  }

  return(wires)
}

binary_to_decimal <- function(bits) sum(bits * 2^(seq_along(bits) - 1))
wires <- process_gates(wires, gates)
z_wires <- wires[grepl('^z\\d+$', names(wires))]
z_bits <- z_wires[order(as.numeric(sub('z', '', names(z_wires))))]

# part 1
binary_to_decimal(z_bits)


# part 2 -- could not solve; this is refactored code from github user cettt
get_val <- function(x) gsub('.* -> ', '', x)
format_wire_id <- function(n) ifelse(n <= 9L, paste0('0', n), as.character(n))
gate_ext <- function(x) regmatches(x, gregexpr('[a-z]{3}', x))[[1]]
gate_ends_with <- function(x) grep(paste0(x, '$'), eq, value = TRUE)
eq <- sapply(gates, function(g) paste(g$inp1, g$op, g$inp2, '->', g$output))
wrong_z <- get_val(eq[!grepl('X', eq) & grepl('z', eq) & !grepl('z45', eq)])

right_z <- vector(mode = 'character')
for (n in sub('z', '', wrong_z)) {
  a <- eq[grepl(paste0('x', n), eq) & grepl(paste0('y', n), eq) & grepl('X', eq)]
  b <- eq[grepl(paste0(get_val(a), '.'), eq) & grepl('X', eq)]
  right_z <- c(right_z, get_val(b))
}

for (i in seq_along(wrong_z)) {
  eq <- gsub(paste0(wrong_z[i], '$'), paste0(right_z[i], '999'), eq)
  eq <- gsub(paste0(right_z[i], '$'), wrong_z[i], eq)
  eq <- gsub('999', '', eq)
}

out <- vector(mode = 'list', length = length(z_wires) - 3)
for (i in seq.int(2, length(z_wires) - 2))  {
  eq_z1 <- grep(paste0('z', format_wire_id(i)), eq, value = TRUE)
  eq_z2 <- sapply(gate_ext(eq_z1), gate_ends_with)
  eq_z2 <- eq_z2[order(!grepl('x\\d{2}', eq_z2))]
  eq_z3 <- sapply(gate_ext(eq_z2[2])[1:2], gate_ends_with)
  eq_z3 <- eq_z3[order(!grepl('x\\d{2}', eq_z3))]
  out[[i - 1]] <- c(eq_z1, eq_z2, eq_z3)
}

non_xor_gates <- names(grep('X', sapply(out, function(z) z[2]),
                            value = TRUE,
                            invert = TRUE))

non_and_gates <- names(grep('A', sapply(out, function(z) z[4]),
                            value = TRUE,
                            invert = TRUE))

paste0(sort(c(wrong_z, right_z, non_xor_gates, non_and_gates)), collapse = ',')

## Day 25

In [ ]:
input <- readLines(get_data('day_25'))

convert_to_height <- function(lines) {
  mat <- do.call(rbind, strsplit(lines[lines != ''], ''))
  heights <- colSums(mat == '#') - 1
  if (all(mat[1, ] == '#')) {
    return(-heights)
  } else {
    return(heights)
  }
}

parse_schematics <- function(input_data) {
  groups <- split(input_data, cumsum(input_data == ''))
  heights <- sapply(groups, convert_to_height)
  locks <- heights[, colSums(heights) > 0]
  keys <- -heights[, colSums(heights) < 0]
  list(locks = locks, keys = keys)
}

count_valid_pairs <- function(locks, keys, max_height = 5) {
  pair_count <- 0
  for (key in seq_len(ncol(keys))) {
    valid <- colSums(keys[, key] + locks <= max_height) == nrow(locks)
    pair_count <- pair_count + sum(valid)
  }
  pair_count
}

schematics <- parse_schematics(input)
count_valid_pairs(schematics$locks, schematics$keys)